In [1]:
import torch
import pandas as pd
import torch.nn as nn
from torch.utils import data
import numpy as np

import os

In [2]:
DATA_DIR = '/Users/aplle/Code/MathModeling/SC/dataset1'
MODEL_DIR = '/Users/aplle/Code/MathModeling/SC/models'
RESULT_DIR = '/Users/aplle/Code/MathModeling/SC/results'

In [8]:
train_df = pd.read_csv(f"{DATA_DIR}/train.csv")
eval_df = pd.read_csv(f"{DATA_DIR}/eval.csv")
test_df = pd.read_csv(f"{DATA_DIR}/test.csv")

POWER_DIVISOR = 1000.0
YEAR_MIN = 2016
YEAR_MAX = 2018

def is_leap(yr:int):
    if yr % 400 == 0:
        return True
    elif yr % 100 == 0:
        return False
    elif yr % 4 == 0:
        return True
    return False

def get_ndays(yr:int):
    return 366 if is_leap(yr) else 365

def preprocess_df(df):
    df.dropna(subset=['Power (kW)'], inplace=True)
    df['Power (kW)'] = df['Power (kW)'] / POWER_DIVISOR
    df.rename(columns={'Power (kW)': 'Power'}, inplace=True)

    df['Days_from_NYD'] = df['Days_from_NYD'] / df['Year'].apply(get_ndays)

    df['Year'] = df['Year'] - YEAR_MIN
    df['Year'] = df['Year'] / (YEAR_MAX - YEAR_MIN)

    df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S')
    df['Time'] = df['Time'].dt.hour * 60 + df['Time'].dt.minute
    df['Time'] = df['Time'] / (24 * 60)

    df.drop(['Day', 'Span'], axis=1, inplace=True)

    df['Month'] = df['Month'] / 12.0

    df['Weekday'] = df['Weekday'] / 6.0

    # df['Region'] = df['Region'].map({
    #     'Commercial': 0,
    #     'Office': 1,
    #     'Public': 2,
    #     'Residential': 3
    # })

    return df

train_df = preprocess_df(train_df)
_train_columns = train_df.columns.copy()

eval_df = preprocess_df(eval_df)
eval_df = eval_df.reindex(columns=_train_columns, fill_value=0)

test_df = preprocess_df(test_df)
test_df = test_df.reindex(columns=_train_columns, fill_value=0)

In [12]:
class GM11:
    def __init__(self):
        self.a = None
        self.b = None
        self.x0 = None

    def fit(self, x):
        """
        x: shape [T], 原始序列
        """
        x = np.asarray(x, dtype=np.float64)
        assert len(x) >= 4, "GM(1,1) requires at least 4 points"

        self.x0 = x.copy()

        # AGO
        x1 = np.cumsum(x)

        # 邻均生成序列
        z1 = 0.5 * (x1[1:] + x1[:-1])

        # 构造 B, Y
        B = np.column_stack((-z1, np.ones_like(z1)))
        Y = x[1:].reshape(-1, 1)

        # 最小二乘估计
        theta = np.linalg.lstsq(B, Y, rcond=None)[0]
        self.a, self.b = theta.flatten()

        return self

    def predict(self, steps=1):
        """
        预测未来 steps 个点
        """
        a, b = self.a, self.b
        x0 = self.x0
        x1_0 = x0[0]

        preds = []
        for k in range(len(x0), len(x0) + steps):
            x1_hat = (x1_0 - b / a) * np.exp(-a * k) + b / a
            x1_prev = (x1_0 - b / a) * np.exp(-a * (k - 1)) + b / a
            preds.append(x1_hat - x1_prev)

        return np.array(preds)

def gm_predict_last(df, region, hist_len):
    """
    用最近 hist_len 个点预测下一步
    """
    series = (
        df[df["Region"] == region]["Power"]
        .to_numpy()
    )

    Y_pred = []
    Y_true = []

    for i in range(hist_len, len(series)):
        x_hist = series[i-hist_len:i]

        model = GM11().fit(x_hist)
        y_pred = model.predict(steps=1)[0]
        Y_pred.append(y_pred)

        y_true = series[i]
        Y_true.append(y_true)

    return Y_pred, Y_true


In [13]:
regions = ['Commercial', 'Office', 'Public', 'Residential']
for region in regions:
    Y_pred, Y_true = gm_predict_last(test_df, region, hist_len=288)
    Y_pred = np.array(Y_pred) * POWER_DIVISOR
    Y_true = np.array(Y_true) * POWER_DIVISOR
    df = pd.DataFrame({'Predicted Power (kW)': Y_pred, 'True Power (kW)': Y_true})
    df.to_csv(os.path.join(RESULT_DIR, f'gm11_{region}_results.csv'), index=False)

    df['Error'] = df['Predicted Power (kW)'] - df['True Power (kW)']
    df['Absolute Error'] = df['Error'].abs()
    mae = df['Absolute Error'].mean()
    rmse = np.sqrt((df['Error'] ** 2).mean())
    mape = (df['Absolute Error'] / df['True Power (kW)']).mean() * 100
    print(f"Region: {region}")
    print(f"MAE: {mae:.2f} kW")
    print(f"RMSE: {rmse:.2f} kW")
    print(f"MAPE: {mape:.2f} %")
    print("")



Region: Commercial
MAE: 247.89 kW
RMSE: 305.86 kW
MAPE: 46.53 %

Region: Office
MAE: 210.05 kW
RMSE: 298.23 kW
MAPE: 62.22 %

Region: Public
MAE: 84.97 kW
RMSE: 108.41 kW
MAPE: 51.12 %

Region: Residential
MAE: 66.38 kW
RMSE: 81.06 kW
MAPE: 24.19 %



In [7]:
test_df

,Time,Power,Year,Month,Weekday,Region,Days_from_NYD
0,0.000000,0.88704,1.0,0.75,0.833333,0,0.665753
1,0.003472,0.88821,1.0,0.75,0.833333,0,0.665753
2,0.006944,0.86266,1.0,0.75,0.833333,0,0.665753
3,0.010417,0.87131,1.0,0.75,0.833333,0,0.665753
4,0.013889,0.83178,1.0,0.75,0.833333,0,0.665753
...,...,...,...,...,...,...,...
141027,0.986111,0.20560,1.0,1.00,0.000000,2,0.997260
141028,0.989583,0.17880,1.0,1.00,0.000000,2,0.997260
141029,0.993056,0.18240,1.0,1.00,0.000000,2,0.997260
141030,0.996528,0.18640,1.0,1.00,0.000000,2,0.997260
